In [1]:
import pandas as pd
import os

# List of your full file paths
file_paths = [
    r"C:\Users\walee\Documents\SMU Work\Capstone Final\ERCOT Wind Integration Reports 2010-2016\ERCOTWindIntegrationReport_2010.xlsx",
    r"C:\Users\walee\Documents\SMU Work\Capstone Final\ERCOT Wind Integration Reports 2010-2016\ERCOTWindIntegrationReport_2011.xlsx",
    r"C:\Users\walee\Documents\SMU Work\Capstone Final\ERCOT Wind Integration Reports 2010-2016\ERCOTWindIntegrationReport_2012.xlsx",
    r"C:\Users\walee\Documents\SMU Work\Capstone Final\ERCOT Wind Integration Reports 2010-2016\ERCOTWindIntegrationReport_2013.xlsx",
    r"C:\Users\walee\Documents\SMU Work\Capstone Final\ERCOT Wind Integration Reports 2010-2016\ERCOTWindIntegrationReport_2014.xlsx",
    r"C:\Users\walee\Documents\SMU Work\Capstone Final\ERCOT Wind Integration Reports 2010-2016\ERCOTWindIntegrationReport_2015.xlsx",
    r"C:\Users\walee\Documents\SMU Work\Capstone Final\ERCOT Wind Integration Reports 2010-2016\ERCOTWindIntegrationReport_Jan2016.csv"
]

# Initialize an empty list to hold DataFrames
df_list = []

# Loop through and load each file
for path in file_paths:
    if path.endswith('.xlsx'):
        df = pd.read_excel(path)
    elif path.endswith('.csv'):
        df = pd.read_csv(path)
    else:
        continue  # skip anything else

    df['Source_File'] = os.path.basename(path)
    df_list.append(df)

# Combine into one big DataFrame
wind_df = pd.concat(df_list, ignore_index=True)

# Preview the data
wind_df.head()

,Date,Peak Load (MW),Peak Load Hour,Wind @ Peak Load (MW),Max Wind Output (MW),Max Wind Time,Penetration @ Max Wind (%),Max Penetration (%),Max Penetration Time,Max Penetration (MW),...,All-Time Max Penetration (%),All-Time Max Penetration Time,Wind @ Record Penetration (MW),Notes,Source File,Source_File,Unnamed: 17,Unnamed: 18,Wind @ Max Penetration (MW),All-Time Max Wind Penetration (%)
0,2010-08-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,ERCOT,ERCOTWindIntegrationReport_2010.xlsx,NaN,NaN,NaN,NaN
1,2010-08-02 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,ERCOT,ERCOTWindIntegrationReport_2010.xlsx,NaN,NaN,NaN,NaN
2,2010-08-03 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,ERCOT,ERCOTWindIntegrationReport_2010.xlsx,NaN,NaN,NaN,NaN
3,2010-08-04 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,ERCOT,ERCOTWindIntegrationReport_2010.xlsx,NaN,NaN,NaN,NaN
4,2010-08-05 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,ERCOT,ERCOTWindIntegrationReport_2010.xlsx,NaN,NaN,NaN,NaN


In [2]:
print(wind_df.columns.tolist())

['Date', 'Peak Load (MW)', 'Peak Load Hour', 'Wind @ Peak Load (MW)', 'Max Wind Output (MW)', 'Max Wind Time', 'Penetration @ Max Wind (%)', 'Max Penetration (%)', 'Max Penetration Time', 'Max Penetration (MW)', '[N/A]', '[N/A].1', 'All-Time Max Wind Output (MW)', 'All-Time Max Output Time', 'All-Time Max Penetration (%)', 'All-Time Max Penetration Time', 'Wind @ Record Penetration (MW)', 'Notes', 'Source File', 'Source_File', 'Unnamed: 17', 'Unnamed: 18', 'Wind @ Max Penetration (MW)', 'All-Time Max Wind Penetration (%)']


In [3]:
# Drop junk columns
junk_cols = ['[N/A]', '[N/A].1', 'Unnamed: 17', 'Unnamed: 18', 'Source File']
wind_df = wind_df.drop(columns=junk_cols)

# Ensure 'Date' is parsed properly
wind_df['Date'] = pd.to_datetime(wind_df['Date'], errors='coerce')

# Check again
wind_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2007 entries, 0 to 2006
Data columns (total 19 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   Date                               2007 non-null   datetime64[ns]
 1   Peak Load (MW)                     1993 non-null   float64       
 2   Peak Load Hour                     1993 non-null   float64       
 3   Wind @ Peak Load (MW)              1993 non-null   float64       
 4   Max Wind Output (MW)               1993 non-null   float64       
 5   Max Wind Time                      1993 non-null   object        
 6   Penetration @ Max Wind (%)         1962 non-null   object        
 7   Max Penetration (%)                86 non-null     object        
 8   Max Penetration Time               55 non-null     object        
 9   Max Penetration (MW)               0 non-null      float64       
 10  All-Time Max Wind Output (MW)      1

In [4]:
core_columns = [
    'Date', 'Peak Load (MW)', 'Peak Load Hour', 'Wind @ Peak Load (MW)',
    'Max Wind Output (MW)', 'Max Wind Time', 'Penetration @ Max Wind (%)',
    'Source_File'
]

wind_df_clean = wind_df[core_columns].copy()

In [5]:
# Calculate curtailment proxy
wind_df_clean['Curtailment Proxy (MW)'] = (
    wind_df_clean['Max Wind Output (MW)'] - wind_df_clean['Wind @ Peak Load (MW)']
)

# Calculate Wind Utilization at Peak Load
wind_df_clean['Wind Utilization % @ Peak'] = (
    wind_df_clean['Wind @ Peak Load (MW)'] / wind_df_clean['Max Wind Output (MW)']
) * 100

# Round to two decimals for easier viewing
wind_df_clean['Curtailment Proxy (MW)'] = wind_df_clean['Curtailment Proxy (MW)'].round(2)
wind_df_clean['Wind Utilization % @ Peak'] = wind_df_clean['Wind Utilization % @ Peak'].round(2)

In [6]:
# Look at a sample to verify
wind_df_clean[['Date', 'Peak Load (MW)', 'Wind @ Peak Load (MW)', 'Max Wind Output (MW)',
               'Curtailment Proxy (MW)', 'Wind Utilization % @ Peak']].head(10)

,Date,Peak Load (MW),Wind @ Peak Load (MW),Max Wind Output (MW),Curtailment Proxy (MW),Wind Utilization % @ Peak
0,2010-08-01,NaN,NaN,NaN,NaN,NaN
1,2010-08-02,NaN,NaN,NaN,NaN,NaN
2,2010-08-03,NaN,NaN,NaN,NaN,NaN
3,2010-08-04,NaN,NaN,NaN,NaN,NaN
4,2010-08-05,NaN,NaN,NaN,NaN,NaN
5,2010-08-06,NaN,NaN,NaN,NaN,NaN
6,2010-08-07,NaN,NaN,NaN,NaN,NaN
7,2010-08-08,NaN,NaN,NaN,NaN,NaN
8,2010-08-09,63467.0,4062.0,4062.0,0.0,100.0
9,2010-08-10,63830.0,3344.0,3344.0,0.0,100.0


In [7]:
import pandas as pd
import os
from glob import glob

# Define file paths
folder_path = r"C:\Users\walee\Documents\SMU Work\Capstone Final\ERCOT Wind Integration Reports 2010-2016"
file_paths = glob(os.path.join(folder_path, "*.xlsx")) + glob(os.path.join(folder_path, "*.csv"))

core_columns = [
    'Date', 'Peak Load (MW)', 'Peak Load Hour', 'Wind @ Peak Load (MW)',
    'Max Wind Output (MW)', 'Max Wind Time', 'Penetration @ Max Wind (%)',
    'Source_File'
]

# Helper function for cleaning
def clean_wind_file(file_path):
    df = pd.read_excel(file_path) if file_path.endswith(".xlsx") else pd.read_csv(file_path)
    df['Source_File'] = os.path.basename(file_path)

    # Drop irrelevant or junk columns
    drop_cols = [col for col in df.columns if 'Unnamed' in col or '[N/A]' in col or 'Source File' in col]
    df.drop(columns=drop_cols, inplace=True, errors='ignore')

    # Fix date
    if 'Date' in df.columns:
        df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

    # Calculate metrics
    df['Curtailment Proxy (MW)'] = (df['Max Wind Output (MW)'] - df['Wind @ Peak Load (MW)']).round(2)
    df['Wind Utilization % @ Peak'] = (
        df['Wind @ Peak Load (MW)'] / df['Max Wind Output (MW)'] * 100
    ).round(2)

    return df[core_columns + ['Curtailment Proxy (MW)', 'Wind Utilization % @ Peak']]


In [8]:
all_years = []

for path in file_paths:
    try:
        df_cleaned = clean_wind_file(path)
        all_years.append(df_cleaned)
    except Exception as e:
        print(f"Error processing {path}: {e}")

# Combine into one big DataFrame
wind_all_df_clean = pd.concat(all_years, ignore_index=True)

# Final check
wind_all_df_clean.info()
wind_all_df_clean.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2007 entries, 0 to 2006
Data columns (total 10 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   Date                        2007 non-null   datetime64[ns]
 1   Peak Load (MW)              1993 non-null   float64       
 2   Peak Load Hour              1993 non-null   float64       
 3   Wind @ Peak Load (MW)       1993 non-null   float64       
 4   Max Wind Output (MW)        1993 non-null   float64       
 5   Max Wind Time               1993 non-null   object        
 6   Penetration @ Max Wind (%)  1962 non-null   object        
 7   Source_File                 2007 non-null   object        
 8   Curtailment Proxy (MW)      1993 non-null   float64       
 9   Wind Utilization % @ Peak   1993 non-null   float64       
dtypes: datetime64[ns](1), float64(6), object(3)
memory usage: 156.9+ KB


,Date,Peak Load (MW),Peak Load Hour,Wind @ Peak Load (MW),Max Wind Output (MW),Max Wind Time,Penetration @ Max Wind (%),Source_File,Curtailment Proxy (MW),Wind Utilization % @ Peak
0,2010-08-01,NaN,NaN,NaN,NaN,NaN,NaN,ERCOTWindIntegrationReport_2010.xlsx,NaN,NaN
1,2010-08-02,NaN,NaN,NaN,NaN,NaN,NaN,ERCOTWindIntegrationReport_2010.xlsx,NaN,NaN
2,2010-08-03,NaN,NaN,NaN,NaN,NaN,NaN,ERCOTWindIntegrationReport_2010.xlsx,NaN,NaN
3,2010-08-04,NaN,NaN,NaN,NaN,NaN,NaN,ERCOTWindIntegrationReport_2010.xlsx,NaN,NaN
4,2010-08-05,NaN,NaN,NaN,NaN,NaN,NaN,ERCOTWindIntegrationReport_2010.xlsx,NaN,NaN


In [9]:
# Save to CSV
wind_all_df_clean.to_csv('cleaned_wind_data_2010_2016.csv', index=False)

In [10]:
(wind_all_df_clean.isnull().sum() / len(wind_all_df_clean) * 100).sort_values(ascending=False)

Penetration @ Max Wind (%)    2.242152
Peak Load (MW)                0.697559
Peak Load Hour                0.697559
Wind @ Peak Load (MW)         0.697559
Max Wind Output (MW)          0.697559
Max Wind Time                 0.697559
Curtailment Proxy (MW)        0.697559
Wind Utilization % @ Peak     0.697559
Date                          0.000000
Source_File                   0.000000
dtype: float64

In [11]:
wind_all_df_clean['Year'] = wind_all_df_clean['Date'].dt.year

In [13]:
missing_by_year = wind_all_df_clean.groupby('Year').apply(lambda x: x.isnull().mean() * 100)
missing_by_year

C:\Users\walee\AppData\Local\Temp\ipykernel_52008\1756355375.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  missing_by_year = wind_all_df_clean.groupby('Year').apply(lambda x: x.isnull().mean() * 100)


,Date,Peak Load (MW),Peak Load Hour,Wind @ Peak Load (MW),Max Wind Output (MW),Max Wind Time,Penetration @ Max Wind (%),Source_File,Curtailment Proxy (MW),Wind Utilization % @ Peak,Year
Year,,,,,,,,,,,
2010,0.0,5.228758,5.228758,5.228758,5.228758,5.228758,5.228758,0.0,5.228758,5.228758,0.0
2011,0.0,0.273973,0.273973,0.273973,0.273973,0.273973,0.273973,0.0,0.273973,0.273973,0.0
2012,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0
2013,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0
2014,0.0,1.377410,1.377410,1.377410,1.377410,1.377410,1.377410,0.0,1.377410,1.377410,0.0
2015,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,8.516484,0.0,0.000000,0.000000,0.0
2016,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0


In [14]:
wind_all_df_clean.describe(include='all')

,Date,Peak Load (MW),Peak Load Hour,Wind @ Peak Load (MW),Max Wind Output (MW),Max Wind Time,Penetration @ Max Wind (%),Source_File,Curtailment Proxy (MW),Wind Utilization % @ Peak,Year
count,2007,1993.000000,1993.000000,1993.000000,1993.000000,1993,1962.0,2007,1993.000000,1993.000000,2007.000000
unique,NaN,NaN,NaN,NaN,NaN,922,1412.0,7,NaN,NaN,NaN
top,NaN,NaN,NaN,NaN,NaN,00:00:00,16.9,ERCOTWindIntegrationReport_2012.xlsx,NaN,NaN,NaN
freq,NaN,NaN,NaN,NaN,NaN,111,7.0,366,NaN,NaN,NaN
mean,2013-04-30 17:10:18.834080768,46014.689413,16.276969,3814.676367,6170.325138,NaN,NaN,NaN,2355.648771,61.568249,2012.815147
min,2010-08-01 00:00:00,30033.000000,7.000000,100.000000,634.000000,NaN,NaN,NaN,0.000000,1.750000,2010.000000
25%,2011-12-15 12:00:00,37103.000000,17.000000,1859.000000,4644.000000,NaN,NaN,NaN,486.000000,35.640000,2011.000000
50%,2013-04-30 00:00:00,43654.000000,17.000000,3614.000000,6160.000000,NaN,NaN,NaN,2212.000000,60.850000,2013.000000
75%,2014-09-14 12:00:00,54834.000000,19.000000,5468.000000,7629.000000,NaN,NaN,NaN,3744.000000,91.570000,2014.000000
max,2016-01-31 00:00:00,69785.000000,22.000000,12403.000000,13883.000000,NaN,NaN,NaN,11642.000000,100.000000,2016.000000


In [15]:
# -- Ensure datetime
wind_all_df_clean['Date'] = pd.to_datetime(wind_all_df_clean['Date'])

# -- Temporal Features
wind_all_df_clean['Month'] = wind_all_df_clean['Date'].dt.month
wind_all_df_clean['DayOfWeek'] = wind_all_df_clean['Date'].dt.dayofweek
wind_all_df_clean['Is_Weekend'] = wind_all_df_clean['DayOfWeek'].isin([5, 6]).astype(int)

# -- Load-Wind Ratio
wind_all_df_clean['Load-Wind Ratio'] = wind_all_df_clean['Peak Load (MW)'] / (wind_all_df_clean['Wind @ Peak Load (MW)'] + 1)

# -- High Wind Day (binary flag)
wind_all_df_clean['High Wind Day'] = (wind_all_df_clean['Wind Utilization % @ Peak'] > 80).astype(int)

# -- Curtailment Severity (categorical from proxy)
def categorize_curtailment(mw):
    if pd.isna(mw): return 'Unknown'
    elif mw < 500: return 'None'
    elif mw < 2000: return 'Low'
    elif mw < 5000: return 'Medium'
    else: return 'High'

wind_all_df_clean['Curtailment Severity'] = wind_all_df_clean['Curtailment Proxy (MW)'].apply(categorize_curtailment)

In [16]:
wind_all_df_clean[['Date', 'Month', 'DayOfWeek', 'Is_Weekend',
                   'Load-Wind Ratio', 'High Wind Day', 
                   'Curtailment Proxy (MW)', 'Curtailment Severity']].head(10)

,Date,Month,DayOfWeek,Is_Weekend,Load-Wind Ratio,High Wind Day,Curtailment Proxy (MW),Curtailment Severity
0,2010-08-01,8,6,1,NaN,0,NaN,Unknown
1,2010-08-02,8,0,0,NaN,0,NaN,Unknown
2,2010-08-03,8,1,0,NaN,0,NaN,Unknown
3,2010-08-04,8,2,0,NaN,0,NaN,Unknown
4,2010-08-05,8,3,0,NaN,0,NaN,Unknown
5,2010-08-06,8,4,0,NaN,0,NaN,Unknown
6,2010-08-07,8,5,1,NaN,0,NaN,Unknown
7,2010-08-08,8,6,1,NaN,0,NaN,Unknown
8,2010-08-09,8,0,0,15.620724,1,0.0,None
9,2010-08-10,8,1,0,19.082212,1,0.0,None


In [17]:
# Check severity distribution
print(wind_all_df_clean['Curtailment Severity'].value_counts())

# Check high wind days
print(wind_all_df_clean['High Wind Day'].value_counts())

# Basic stats for ratio
print(wind_all_df_clean['Load-Wind Ratio'].describe())

Curtailment Severity
Medium     825
None       503
Low        444
High       221
Unknown     14
Name: count, dtype: int64
High Wind Day
0    1322
1     685
Name: count, dtype: int64
count    1993.000000
mean       22.338498
std        27.765806
min         2.941056
25%         7.774065
50%        13.279515
75%        25.275442
max       373.861386
Name: Load-Wind Ratio, dtype: float64


In [18]:
wind_all_df_clean[['Month', 'DayOfWeek', 'Load-Wind Ratio', 'High Wind Day', 'Curtailment Severity']].isnull().sum()

Month                    0
DayOfWeek                0
Load-Wind Ratio         14
High Wind Day            0
Curtailment Severity     0
dtype: int64

In [19]:
core_years = [2012, 2013, 2014, 2015]

key_columns = [
    'Peak Load (MW)',
    'Wind @ Peak Load (MW)',
    'Max Wind Output (MW)',
    'Curtailment Proxy (MW)',
    'Wind Utilization % @ Peak'
]

# Keep only rows for 2012–2015 and drop those with nulls in key columns
wind_df_marl_ready = wind_all_df_clean[
    (wind_all_df_clean['Year'].isin(core_years)) &
    (wind_all_df_clean[key_columns].notnull().all(axis=1))
].copy()

In [20]:
print(wind_df_marl_ready['Year'].value_counts())
print(wind_df_marl_ready.isnull().sum())

Year
2012    366
2013    365
2015    364
2014    358
Name: count, dtype: int64
Date                           0
Peak Load (MW)                 0
Peak Load Hour                 0
Wind @ Peak Load (MW)          0
Max Wind Output (MW)           0
Max Wind Time                  0
Penetration @ Max Wind (%)    31
Source_File                    0
Curtailment Proxy (MW)         0
Wind Utilization % @ Peak      0
Year                           0
Month                          0
DayOfWeek                      0
Is_Weekend                     0
Load-Wind Ratio                0
High Wind Day                  0
Curtailment Severity           0
dtype: int64


In [21]:
wind_df_marl_ready.to_csv("wind_data_marl_ready_2012_2015.csv", index=False)

In [65]:
from sklearn.preprocessing import MinMaxScaler

# Select features to normalize
features_to_scale = [
    'Peak Load (MW)', 'Max Wind Output (MW)', 'Wind @ Peak Load (MW)',
    'Curtailment Proxy (MW)', 'Penetration @ Max Wind (%)', 'Wind Utilization % @ Peak'
]

# Initialize the scaler
scaler = MinMaxScaler()

# Fit and transform the selected columns
df[features_to_scale] = scaler.fit_transform(df[features_to_scale])

# OPTIONAL: Display a quick check
print(df[features_to_scale].describe())


       Peak Load (MW)  Max Wind Output (MW)  Wind @ Peak Load (MW)  \
count     1453.000000           1453.000000            1453.000000   
mean         0.401074              0.443961               0.281936   
std          0.252538              0.175749               0.195401   
min          0.000000              0.000000               0.000000   
25%          0.176381              0.325987               0.124766   
50%          0.350168              0.451430               0.236365   
75%          0.616621              0.567515               0.404535   
max          1.000000              1.000000               1.000000   

       Curtailment Proxy (MW)  Penetration @ Max Wind (%)  \
count             1453.000000                 1422.000000   
mean                 0.314721                    0.428590   
std                  0.195287                    0.197544   
min                  0.000000                    0.000000   
25%                  0.155973                    0.283520   
50%

In [22]:
!pip install gym numpy pandas

     ---------------------------------------- 0.0/721.7 kB ? eta -:--:--
     -------------------------------------- 721.7/721.7 kB 7.4 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for gym: filename=gym-0.26.2-py3-none-any.whl size=827741 sha256=5b9148e04466029bd504116232fb0da33ee37359bf9dfdd0ba04c797c6eaadf1
  Stored in directory: c:\users\walee\appdata\local\pip\cache\wheels\95\51\6c\9bb05ebbe7c5cb8171dfaa3611f32622ca4658d53f31c79077
Successfully built gym


In [61]:
import gym
import numpy as np
from gym import spaces

class WindGridEnv(gym.Env):
    def __init__(self, df, agent_ids):
        super(WindGridEnv, self).__init__()
        self.df = df.reset_index(drop=True)
        self.agent_ids = agent_ids
        self.current_step = 0
        self.max_steps = len(df)

        #  Define all columns used as observation features
        self.state_columns = [
            'Peak Load (MW)',
            'Max Wind Output (MW)',
            'Wind @ Peak Load (MW)',
            'Curtailment Proxy (MW)',
            'Penetration @ Max Wind (%)',
            'Wind Utilization % @ Peak'
        ]

        self.state_size = len(self.state_columns)
        self.action_space = spaces.Discrete(3)  # 0 = do nothing, 1 = curtail, 2 = store

    def reset(self):
        self.current_step = 0
        row = self.df.iloc[self.current_step]
        obs = {
            agent: np.array([row[col] for col in self.state_columns], dtype=np.float32)
            for agent in self.agent_ids
        }
        return obs

    def step(self, actions):
        rewards = {}
        done = False
        info = {}

        if self.current_step >= self.max_steps - 1:
            done = True
            return self.reset(), {a: 0 for a in self.agent_ids}, done, info

        row = self.df.iloc[self.current_step]

        for agent_id, action in actions.items():
            wind = row['Wind @ Peak Load (MW)']
            peak = row['Peak Load (MW)']
            curtailment = row['Curtailment Proxy (MW)']
            utilization = row['Wind Utilization % @ Peak']

            # Reward logic
            if action == 0:
                reward = -curtailment
            elif action == 1:
                reward = -abs(curtailment - 0.1 * peak)
            elif action == 2:
                reward = utilization * 2 - curtailment
            else:
                reward = -10  # invalid action

            rewards[agent_id] = reward

        self.current_step += 1
        next_row = self.df.iloc[self.current_step]
        obs = {
            agent: np.array([next_row[col] for col in self.state_columns], dtype=np.float32)
            for agent in self.agent_ids
        }

        return obs, rewards, done, info


In [62]:
df = pd.read_csv("wind_data_marl_ready_2012_2015.csv")

env = WindGridEnv(df, agent_ids=['Substation_1', 'Substation_2'])

obs = env.reset()

for _ in range(5):  # test first 5 steps
    actions = {agent: env.action_space.sample() for agent in env.agent_ids}
    obs, rewards, done, info = env.step(actions)
    print(f"Step: {env.current_step}, Rewards: {rewards}")
    if done:
        break

Step: 1, Rewards: {'Substation_1': -1786.7999999999997, 'Substation_2': -5038.0}
Step: 2, Rewards: {'Substation_1': -1650.5, 'Substation_2': -1650.5}
Step: 3, Rewards: {'Substation_1': -804.62, 'Substation_2': -804.62}
Step: 4, Rewards: {'Substation_1': -1614.38, 'Substation_2': -1761.0}
Step: 5, Rewards: {'Substation_1': -5951.78, 'Substation_2': -5951.78}


In [39]:
!pip install torch

In [63]:
import torch
import torch.nn as nn
import torch.optim as optim
import random
import numpy as np
from collections import deque

class DQNAgent:
    def __init__(self, state_size, action_size, name='Substation_1'):
        self.state_size = state_size
        self.action_size = action_size
        self.name = name

        self.memory = deque(maxlen=2000)
        self.gamma = 0.95      # Discount rate
        self.epsilon = 1.0     # Exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001

        self.model = self._build_model()

    def _build_model(self):
        return nn.Sequential(
            nn.Linear(self.state_size, 24),
            nn.ReLU(),
            nn.Linear(24, 24),
            nn.ReLU(),
            nn.Linear(24, self.action_size)
        )

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        state = torch.FloatTensor(state).unsqueeze(0)
        q_values = self.model(state)
        return torch.argmax(q_values).item()

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def replay(self, batch_size=32):
        if len(self.memory) < batch_size:
            return

        minibatch = random.sample(self.memory, batch_size)
        criterion = nn.MSELoss()
        optimizer = optim.Adam(self.model.parameters(), lr=self.learning_rate)

        for state, action, reward, next_state, done in minibatch:
            state = torch.FloatTensor(state).unsqueeze(0)
            next_state = torch.FloatTensor(next_state).unsqueeze(0)

            target = reward
            if not done:
                target += self.gamma * torch.max(self.model(next_state)).item()
            target_f = self.model(state).detach().clone()
            target_f[0][action] = target

            # Train
            optimizer.zero_grad()
            output = self.model(state)
            loss = criterion(output, target_f)
            loss.backward()
            optimizer.step()

        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

In [58]:
class MultiAgentTrainer:
    def __init__(self, env, agent_ids, state_size, action_size):
        self.env = env
        self.agent_ids = agent_ids
        self.agents = {
            agent_id: DQNAgent(state_size, action_size)
            for agent_id in agent_ids
        }

    def train(self, episodes=100):
        for episode in range(episodes):
            states = self.env.reset()
            total_rewards = {agent: 0 for agent in self.agent_ids}
            done = False

            while not done:
                actions = {
                    agent: self.agents[agent].act(states[agent])
                    for agent in self.agent_ids
                }

                next_states, rewards, done, _ = self.env.step(actions)

                for agent in self.agent_ids:
                    self.agents[agent].remember(
                        states[agent],
                        actions[agent],
                        rewards[agent],
                        next_states[agent],
                        done
                    )
                    self.agents[agent].replay()

                    total_rewards[agent] += rewards[agent]

                states = next_states

            print(f"Episode {episode+1} - Rewards: {total_rewards}")

In [66]:
# Load the data
df = pd.read_csv("wind_data_marl_ready_2012_2015.csv")

# Define agent IDs (representing substations)
agent_ids = ['Substation_1', 'Substation_2']

# Create environment instance
env = WindGridEnv(df, agent_ids=agent_ids)

# Define state and action sizes (shared across agents)
state_size = env.state_size
action_size = env.action_space.n

# Initialize trainer
trainer = MultiAgentTrainer(env, agent_ids, state_size, action_size)

# Train agents (start small to verify functionality)
trainer.train(episodes=10)

Episode 1 - Rewards: {'Substation_1': -2857222.7199999997, 'Substation_2': -2862212.44}
Episode 2 - Rewards: {'Substation_1': -4273784.000000001, 'Substation_2': -4271501.22}
Episode 3 - Rewards: {'Substation_1': -4280377.66, 'Substation_2': -4273593.0}
Episode 4 - Rewards: {'Substation_1': -4277437.98, 'Substation_2': -4272324.399999999}
Episode 5 - Rewards: {'Substation_1': -4276182.6, 'Substation_2': -4296005.74}
Episode 6 - Rewards: {'Substation_1': -4286042.34, 'Substation_2': -4283032.62}
Episode 7 - Rewards: {'Substation_1': -4277214.56, 'Substation_2': -4271605.84}
Episode 8 - Rewards: {'Substation_1': -4267317.800000001, 'Substation_2': -4273749.46}
Episode 9 - Rewards: {'Substation_1': -4271676.24, 'Substation_2': -4276565.779999999}
Episode 10 - Rewards: {'Substation_1': -4274467.96, 'Substation_2': -4267695.779999999}


In [43]:
import pandas as pd

file_path = r"C:\Users\walee\Documents\SMU Work\Capstone Final\ERCOT Wind Integration Reports 2010-2016\cleaned_wind_data_2010_2016.csv"
df = pd.read_csv(file_path, parse_dates=["Date"])

df.head()

,Date,Peak Load (MW),Peak Load Hour,Wind @ Peak Load (MW),Max Wind Output (MW),Max Wind Time,Penetration @ Max Wind (%),Source_File,Curtailment Proxy (MW),Wind Utilization % @ Peak
0,2010-08-01,NaN,NaN,NaN,NaN,NaN,NaN,ERCOTWindIntegrationReport_2010.xlsx,NaN,NaN
1,2010-08-02,NaN,NaN,NaN,NaN,NaN,NaN,ERCOTWindIntegrationReport_2010.xlsx,NaN,NaN
2,2010-08-03,NaN,NaN,NaN,NaN,NaN,NaN,ERCOTWindIntegrationReport_2010.xlsx,NaN,NaN
3,2010-08-04,NaN,NaN,NaN,NaN,NaN,NaN,ERCOTWindIntegrationReport_2010.xlsx,NaN,NaN
4,2010-08-05,NaN,NaN,NaN,NaN,NaN,NaN,ERCOTWindIntegrationReport_2010.xlsx,NaN,NaN


In [44]:
# Add time-based features
df['Hour'] = df['Date'].dt.hour
df['DayOfWeek'] = df['Date'].dt.dayofweek
df['Is_Weekend'] = df['DayOfWeek'].isin([5, 6]).astype(int)

#  Drop unused columns or reorder
df = df.drop(columns=['DayOfWeek'])

df.head()

,Date,Peak Load (MW),Peak Load Hour,Wind @ Peak Load (MW),Max Wind Output (MW),Max Wind Time,Penetration @ Max Wind (%),Source_File,Curtailment Proxy (MW),Wind Utilization % @ Peak,Hour,Is_Weekend
0,2010-08-01,NaN,NaN,NaN,NaN,NaN,NaN,ERCOTWindIntegrationReport_2010.xlsx,NaN,NaN,0,1
1,2010-08-02,NaN,NaN,NaN,NaN,NaN,NaN,ERCOTWindIntegrationReport_2010.xlsx,NaN,NaN,0,0
2,2010-08-03,NaN,NaN,NaN,NaN,NaN,NaN,ERCOTWindIntegrationReport_2010.xlsx,NaN,NaN,0,0
3,2010-08-04,NaN,NaN,NaN,NaN,NaN,NaN,ERCOTWindIntegrationReport_2010.xlsx,NaN,NaN,0,0
4,2010-08-05,NaN,NaN,NaN,NaN,NaN,NaN,ERCOTWindIntegrationReport_2010.xlsx,NaN,NaN,0,0


In [45]:
print("Data shape:", df.shape)
print("Missing values:\n", df.isna().sum())

Data shape: (2007, 12)
Missing values:
 Date                           0
Peak Load (MW)                14
Peak Load Hour                14
Wind @ Peak Load (MW)         14
Max Wind Output (MW)          14
Max Wind Time                 14
Penetration @ Max Wind (%)    45
Source_File                    0
Curtailment Proxy (MW)        14
Wind Utilization % @ Peak     14
Hour                           0
Is_Weekend                     0
dtype: int64


In [46]:
# Define essential columns for the model
essential_columns = [
    'Peak Load (MW)',
    'Peak Load Hour',
    'Wind @ Peak Load (MW)',
    'Max Wind Output (MW)',
    'Curtailment Proxy (MW)'
]

# Drop rows with missing essential data
df_cleaned = df.dropna(subset=essential_columns)

print("After cleaning:")
print("Shape:", df_cleaned.shape)
print("Missing:\n", df_cleaned.isna().sum())

After cleaning:
Shape: (1993, 12)
Missing:
 Date                           0
Peak Load (MW)                 0
Peak Load Hour                 0
Wind @ Peak Load (MW)          0
Max Wind Output (MW)           0
Max Wind Time                  0
Penetration @ Max Wind (%)    31
Source_File                    0
Curtailment Proxy (MW)         0
Wind Utilization % @ Peak      0
Hour                           0
Is_Weekend                     0
dtype: int64


In [48]:
df_cleaned.loc[:, 'Penetration @ Max Wind (%)'] = df_cleaned['Penetration @ Max Wind (%)'].fillna(0)

In [67]:
# These features will now define each agent's observation state
self.state_columns = [
    'Peak Load (MW)',
    'Max Wind Output (MW)',
    'Wind @ Peak Load (MW)',
    'Curtailment Proxy (MW)',
    'Penetration @ Max Wind (%)',
    'Wind Utilization % @ Peak'
]
self.state_size = len(self.state_columns)

NameError: name 'self' is not defined